In [168]:
from datetime import datetime
from fractions import Fraction

import configparser
import pandas as pd

In [169]:
config = configparser.ConfigParser()
config.read('config.ini')
last_paid = datetime.strptime(config['DEFAULT']['last_paid'], '%Y-%m-%d')
mint_analysis = datetime.strptime(config['DEFAULT']['mint_analysis'], '%Y-%m-%d')

In [170]:
chase = pd.read_csv('ChaseCC.csv')
citi = pd.read_csv('CitiCC.csv')
mint = pd.read_csv('Mint.csv')

In [171]:
chase['trans_date'] = pd.to_datetime(chase['Transaction Date'])
citi['trans_date'] = pd.to_datetime(citi['Date'])
mint['trans_date'] = pd.to_datetime(mint['Date'])

In [172]:
chase = chase[chase['trans_date'] > last_paid]
chase = chase[chase['Type'] != 'Payment']
citi = citi[citi['trans_date'] > last_paid]
citi = citi[~citi['Description'].str.contains('PAYMENT, THANK YOU')]
mint = mint[mint['trans_date'] > mint_analysis]

In [173]:
chase['amt'] = chase['Amount'] * -1

In [174]:
citi.loc[(citi['Debit'].notnull()), 'amt'] = citi['Debit']
citi.loc[(citi['Credit'].notnull()), 'amt'] = citi['Credit']

In [175]:
mint.loc[(mint['Transaction Type'] == 'debit'), 'amt'] = mint['Amount']
mint.loc[(mint['Transaction Type'] == 'credit'), 'amt'] = mint['Amount'] * -1

In [176]:
chase = chase[['trans_date', 'amt', 'Description']]
citi = citi[['trans_date', 'amt', 'Description']]
mint = mint[['trans_date', 'amt', 'Category', 'Description', 'Original Description']]

In [177]:
chase.head(5)

,trans_date,amt,Description
0,2020-12-21,47.69,UBER EATS
1,2020-12-16,25.00,TVY*FITNESSYOURWAY
2,2020-12-14,32.66,UBER EATS
3,2020-12-13,42.77,UBER EATS
4,2020-12-10,114.54,SPRINT *WIRELESS


In [178]:
citi.head(5)

,trans_date,amt,Description
0,2020-12-24,-449.99,PAYPAL *B H PHOTO 4029357733 NY Digital Accoun...
1,2020-12-24,6.36,PAYPAL *AMAGICOM AB 35314369001 SWE
2,2020-12-22,115.62,TRADER JOE'S #692 QPS BRENTWOOD MO
3,2020-12-21,10.69,PAYPAL *SPOTIFYUSAI 4029357733 NY Digital Acco...
4,2020-12-19,126.14,Amazon Prime*YG8V21593 8665572820 WA


In [179]:
mint.head(5)

,trans_date,amt,Category,Description,Original Description
0,2020-12-26,-1550.96,Credit Card Payment,PAYMENT THANK YOU,PAYMENT THANK YOU
1,2020-12-24,498.32,Shopping,MICRO CENTER BRNTWD-09 BRENTWOOD MO,MICRO CENTER BRNTWD-09 BRENTWOOD MO
2,2020-12-25,-8.86,Interest Income,Interest Paid,Interest Paid
3,2020-12-24,-449.99,Shopping,B H Photo,PAYPAL *B H PHOTO 4029357733 NY
4,2020-12-24,6.36,Shopping,AMAGICOM AB 35314369001 SWE,PAYPAL *AMAGICOM AB 35314369001 SWE


In [180]:
round(citi['amt'].sum(),2)

1492.11

In [182]:
income_bk = round(float(config['BUDGET']['income_bk_raw']) * float(Fraction(config['BUDGET']['income_bk_mult'])),2)
income_bk

4855.95

In [185]:
mint['Category'].drop_duplicates()

0          Credit Card Payment
1                     Shopping
2              Interest Income
6                     Paycheck
7                    Groceries
8                      Spotify
9                  Restaurants
12                Mobile Phone
19                 Natural Gas
20                         Gym
22                    Transfer
30                    Electric
36              Alcohol & Bars
42                  Gas & Fuel
50                    Internet
53             Mortgage & Rent
59                        Taxi
65               Misc Expenses
86            One Time Expense
94                     Parking
97                   Financial
130                TV/Internet
164              Entertainment
173                Water/Sewer
191               Work Expense
355               Gas/Electric
385             Auto Insurance
487                     Travel
526                    ATM Fee
531                       Hair
629               Property Tax
636                Furnishings
643     

# TO-Do's
- Store budget information
- Create dictionary of Mint categories to more general categories
- Create calendar of how to predict upcoming paychecks
- Make list/calendar of upcoming expenses
- Calculate remaining budget based off upcoming expenses
- Create budget analysis (regression and past month)
- Create graphs and visuals